# Lese data frå Pureservice
Først definerer eg litt rusk:

In [5]:
import requests
import json
import os
import pandas as pd
from bs4 import BeautifulSoup


def strip_html(html_string):
    soup = BeautifulSoup(html_string, "html.parser")
    return soup.get_text()

baseurl = "https://hvl.pureservice.com/agent/api"

PStoken = os.environ["PStoken"]
headers = {'X-Authorization-Key': PStoken, 'Content-Type': 'application/vnd.api+json'}


Så starter eg på den første saken om Canvas i august 2024 (som framleis er open); det er sak nr 393683. Så hentar eg 500 om gangen til eg har henta alle sakane fram til i dag. For kvar sak sjekker eg om den handlar om Canvas (`assignedDepartment` er 13), og så samler eg alle dei sakene i ei svær dataramme.

In [6]:
start = 393683
antal_i_kvar = 500
antal_søk = 100
dataliste = []
for i in range(antal_søk):
    url = f"{baseurl}/ticket/?start={start + i * antal_i_kvar}&limit={antal_i_kvar}"
    response = requests.request("GET", url, headers=headers)
    print(f"{start + i * antal_i_kvar}-{start + (i + 1) * antal_i_kvar - 1} => {response.status_code}")
    if response.status_code == 200:
        for ticket in response.json()['tickets']:
            if ticket['links']['assignedDepartment']['id'] == 13:
                dataliste.append(pd.DataFrame([ticket]))
alledata = pd.concat(dataliste)

393683-394182 => 200
394183-394682 => 200
394683-395182 => 200
395183-395682 => 200
395683-396182 => 200
396183-396682 => 200
396683-397182 => 200
397183-397682 => 200
397683-398182 => 200
398183-398682 => 200
398683-399182 => 200
399183-399682 => 200
399683-400182 => 200
400183-400682 => 200
400683-401182 => 200
401183-401682 => 200
401683-402182 => 200
402183-402682 => 200
402683-403182 => 200
403183-403682 => 200
403683-404182 => 200
404183-404682 => 200
404683-405182 => 200
405183-405682 => 200
405683-406182 => 200
406183-406682 => 200
406683-407182 => 200
407183-407682 => 200
407683-408182 => 200
408183-408682 => 200
408683-409182 => 200
409183-409682 => 200
409683-410182 => 200
410183-410682 => 200
410683-411182 => 200
411183-411682 => 200
411683-412182 => 200
412183-412682 => 200
412683-413182 => 200
413183-413682 => 200
413683-414182 => 200
414183-414682 => 200
414683-415182 => 200
415183-415682 => 200
415683-416182 => 200
416183-416682 => 200
416683-417182 => 200
417183-417682

In [7]:
alledata.to_csv('alledata.csv', index=False)

Så kan eg hente ut divere informasjon; eg må vel sjekke

1. E-post (kven er tilsett og kven er student og kven sender frå privat e-post)
2. "Subject" (i den grad det er interessant)
3. "Description" er minst like viktig; den stripper eg med funksjonen strip_html()

Først sjekker eg alle felt eg får ut:

In [4]:
alledata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 0 to 0
Data columns (total 53 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   links                 21 non-null     object
 1   solution              19 non-null     object
 2   emailAddress          21 non-null     object
 3   reopenedCount         1 non-null      object
 4   responded             21 non-null     object
 5   resolved              19 non-null     object
 6   reopened              1 non-null      object
 7   closed                19 non-null     object
 8   userWaitedSince       0 non-null      object
 9   userReminded          0 non-null      object
 10  pendingUserSince      2 non-null      object
 11  isMarkedForDeletion   21 non-null     bool  
 12  totalTimelogMinutes   21 non-null     int64 
 13  assignedDepartmentId  21 non-null     int64 
 14  transferComment       0 non-null      object
 15  priorityId            21 non-null     int64 
 16

Så ser eg kor mange saker som er opne (`resolved == None`)

In [24]:
alledata[alledata['resolved'].isna()]

,links,solution,emailAddress,reopenedCount,responded,resolved,reopened,closed,userWaitedSince,userReminded,...,cf_6,cf_7,cf_8,cf_9,id,created,modified,createdById,modifiedById,coordinates
0,"{'assignedDepartment': {'id': 13}, 'priority':...",None,svein.gunnar.sjotun@hvl.no,None,2024-08-05T07:58:34.3582081+00:00,None,None,None,None,None,...,None,None,None,None,400275,2024-08-02T10:39:56.5169800+00:00,2024-08-05T07:58:34.4227730+00:00,3,960,None
0,"{'assignedDepartment': {'id': 13}, 'priority':...",None,camilla.fluge.bortheim@hvl.no,None,2024-08-05T10:29:30.8137610+00:00,None,None,None,None,None,...,None,None,None,None,400645,2024-08-05T07:23:57.5165405+00:00,2024-08-05T10:29:30.8844231+00:00,3,960,None
0,"{'assignedDepartment': {'id': 13}, 'priority':...",None,ida.linn.johnsen.enerstvedt@hvl.no,None,2024-08-05T19:20:08.2587933+00:00,None,None,None,2024-08-06T11:48:29.1670091+00:00,None,...,None,None,None,None,400868,2024-08-05T10:49:57.2211089+00:00,2024-08-06T11:48:29.1690292+00:00,3,3,None
0,"{'assignedDepartment': {'id': 13}, 'priority':...",None,mona.oynes@hvl.no,1,2024-08-06T06:52:04.4522347+00:00,None,2024-08-07T06:06:16.3517792+00:00,None,None,None,...,None,None,None,None,400987,2024-08-05T12:47:29.1708772+00:00,2024-08-07T06:09:26.1362433+00:00,3,960,None
0,"{'assignedDepartment': {'id': 13}, 'priority':...",None,therese.stol@hotmail.com,None,2024-08-05T20:03:23.9804452+00:00,None,None,None,2024-08-06T11:22:57.8253644+00:00,None,...,None,None,None,None,400996,2024-08-05T12:59:27.2920754+00:00,2024-08-06T11:22:57.8274419+00:00,3,3,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"{'assignedDepartment': {'id': 13}, 'priority':...",None,kjersti.hovland@hvl.no,None,2025-02-21T12:37:45.8872156+00:00,None,None,None,2025-02-21T14:24:54.9518906+00:00,None,...,None,None,None,None,444177,2025-02-21T12:31:26.0683929+00:00,2025-02-21T14:24:54.9578264+00:00,3,3,None
0,"{'assignedDepartment': {'id': 13}, 'priority':...",None,angunn.trovag.mjos@hvl.no,None,2025-02-24T11:22:19.5207428+00:00,None,None,None,2025-02-24T11:23:55.0270000+00:00,None,...,None,None,None,None,444355,2025-02-24T08:21:29.6471598+00:00,2025-02-24T11:51:15.5839671+00:00,3,960,None
0,"{'assignedDepartment': {'id': 13}, 'priority':...",None,tove.sandvoll.vee@hvl.no,None,2025-02-24T08:51:12.1733153+00:00,None,None,None,None,None,...,None,None,None,None,444362,2025-02-24T08:30:52.7853845+00:00,2025-02-24T08:58:27.7450538+00:00,3,377,None
0,"{'assignedDepartment': {'id': 13}, 'priority':...",None,knut.magne.aanestad@hvl.no,None,2025-02-25T06:31:55.6063812+00:00,None,None,None,None,None,...,None,None,None,None,444477,2025-02-24T11:45:39.5816303+00:00,2025-02-25T06:33:04.8441254+00:00,3,377,None


Det er altså 161 uløyste saker av i alt 1223 sidan august 2024.

Det neste eg kan gjere er å få ut `subject` og `description` (utan HTML-kode), og så kanskje `userId` (for å finne fakultet).

In [9]:
canvas = alledata[['emailAddress', 'responded', 'resolved', 'subject']]
canvas['innhald'] = alledata['description'].apply(strip_html)
canvas.to_csv("canvas_saker.csv", index=False)

C:\Users\AKV\AppData\Local\Temp\ipykernel_29624\2793442992.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canvas['innhald'] = alledata['description'].apply(strip_html)


## Leit etter spesielle saker
Nedanfor her leiter eg etter spesielle saker; det er eit reint kladdeområde.

In [18]:
alledata[alledata['emailAddress'] == 'kirsti.lysaker@hvl.no']

,links,solution,emailAddress,reopenedCount,responded,resolved,reopened,closed,userWaitedSince,userReminded,...,cf_6,cf_7,cf_8,cf_9,id,created,modified,createdById,modifiedById,coordinates
0,"{'assignedDepartment': {'id': 13}, 'priority':...",<p>Det har gått meir enn tre veker utan svar f...,kirsti.lysaker@hvl.no,None,2024-08-09T06:37:39.4003669+00:00,2024-09-26T12:57:28.5830027+00:00,None,2024-10-03T13:01:18.6427581+00:00,None,None,...,None,None,None,None,402830,2024-08-08T12:46:27.3925312+00:00,2024-10-03T13:01:19.0079069+00:00,3,3,None
0,"{'assignedDepartment': {'id': 13}, 'priority':...",<p>Dette er ein dum ting som er gjort i progra...,kirsti.lysaker@hvl.no,None,2024-09-12T09:53:28.6768789+00:00,2024-09-12T09:53:28.6768789+00:00,None,2024-09-19T10:00:44.7566024+00:00,None,None,...,None,None,None,None,413980,2024-09-12T08:55:57.6462913+00:00,2024-09-19T10:00:45.0399032+00:00,3,3,None


In [20]:
url = f'{baseurl}/ticket/{402830}/requestNumber/'
response = requests.request("GET", url, headers=headers)

In [26]:
pd.DataFrame(response.json()['tickets'][0])

,links,solution,emailAddress,reopenedCount,responded,resolved,reopened,closed,userWaitedSince,userReminded,...,cf_6,cf_7,cf_8,cf_9,id,created,modified,createdById,modifiedById,coordinates
assignedDepartment,{'id': 30},<p>-</p>,aman.2011@live.no,None,2024-08-22T08:49:18.4750138+00:00,2024-09-17T13:41:04.9340274+00:00,None,2024-09-24T13:51:04.0581715+00:00,None,None,...,None,None,None,None,408407,2024-08-22T08:26:55.2471278+00:00,2024-09-24T13:51:04.1637439+00:00,3,3,None
priority,{'id': 4},<p>-</p>,aman.2011@live.no,None,2024-08-22T08:49:18.4750138+00:00,2024-09-17T13:41:04.9340274+00:00,None,2024-09-24T13:51:04.0581715+00:00,None,None,...,None,None,None,None,408407,2024-08-22T08:26:55.2471278+00:00,2024-09-24T13:51:04.1637439+00:00,3,3,None
ticketType,{'id': 5},<p>-</p>,aman.2011@live.no,None,2024-08-22T08:49:18.4750138+00:00,2024-09-17T13:41:04.9340274+00:00,None,2024-09-24T13:51:04.0581715+00:00,None,None,...,None,None,None,None,408407,2024-08-22T08:26:55.2471278+00:00,2024-09-24T13:51:04.1637439+00:00,3,3,None
user,{'id': 105593},<p>-</p>,aman.2011@live.no,None,2024-08-22T08:49:18.4750138+00:00,2024-09-17T13:41:04.9340274+00:00,None,2024-09-24T13:51:04.0581715+00:00,None,None,...,None,None,None,None,408407,2024-08-22T08:26:55.2471278+00:00,2024-09-24T13:51:04.1637439+00:00,3,3,None
assignedTeam,{'id': 71},<p>-</p>,aman.2011@live.no,None,2024-08-22T08:49:18.4750138+00:00,2024-09-17T13:41:04.9340274+00:00,None,2024-09-24T13:51:04.0581715+00:00,None,None,...,None,None,None,None,408407,2024-08-22T08:26:55.2471278+00:00,2024-09-24T13:51:04.1637439+00:00,3,3,None
assignedAgent,{'id': 129239},<p>-</p>,aman.2011@live.no,None,2024-08-22T08:49:18.4750138+00:00,2024-09-17T13:41:04.9340274+00:00,None,2024-09-24T13:51:04.0581715+00:00,None,None,...,None,None,None,None,408407,2024-08-22T08:26:55.2471278+00:00,2024-09-24T13:51:04.1637439+00:00,3,3,None
respondedBy,{'id': 129239},<p>-</p>,aman.2011@live.no,None,2024-08-22T08:49:18.4750138+00:00,2024-09-17T13:41:04.9340274+00:00,None,2024-09-24T13:51:04.0581715+00:00,None,None,...,None,None,None,None,408407,2024-08-22T08:26:55.2471278+00:00,2024-09-24T13:51:04.1637439+00:00,3,3,None
resolvedBy,{'id': 129239},<p>-</p>,aman.2011@live.no,None,2024-08-22T08:49:18.4750138+00:00,2024-09-17T13:41:04.9340274+00:00,None,2024-09-24T13:51:04.0581715+00:00,None,None,...,None,None,None,None,408407,2024-08-22T08:26:55.2471278+00:00,2024-09-24T13:51:04.1637439+00:00,3,3,None
reopenedBy,None,<p>-</p>,aman.2011@live.no,None,2024-08-22T08:49:18.4750138+00:00,2024-09-17T13:41:04.9340274+00:00,None,2024-09-24T13:51:04.0581715+00:00,None,None,...,None,None,None,None,408407,2024-08-22T08:26:55.2471278+00:00,2024-09-24T13:51:04.1637439+00:00,3,3,None
closedBy,{'id': 3},<p>-</p>,aman.2011@live.no,None,2024-08-22T08:49:18.4750138+00:00,2024-09-17T13:41:04.9340274+00:00,None,2024-09-24T13:51:04.0581715+00:00,None,None,...,None,None,None,None,408407,2024-08-22T08:26:55.2471278+00:00,2024-09-24T13:51:04.1637439+00:00,3,3,None


In [22]:
response.content

b'{"tickets":[{"links":{"assignedDepartment":{"id":30},"priority":{"id":4},"ticketType":{"id":5},"user":{"id":105593},"assignedTeam":{"id":71},"assignedAgent":{"id":129239},"respondedBy":{"id":129239},"resolvedBy":{"id":129239},"reopenedBy":null,"closedBy":{"id":3},"form":null,"archiveMetaData":null,"source":{"id":2},"status":{"id":8},"category1":{"id":394},"category2":null,"category3":null,"requestType":{"id":1},"createdBy":{"id":3},"modifiedBy":{"id":3}},"solution":"<p>-</p>","emailAddress":"aman.2011@live.no","reopenedCount":null,"responded":"2024-08-22T08:49:18.4750138+00:00","resolved":"2024-09-17T13:41:04.9340274+00:00","reopened":null,"closed":"2024-09-24T13:51:04.0581715+00:00","userWaitedSince":null,"userReminded":null,"pendingUserSince":null,"isMarkedForDeletion":false,"totalTimelogMinutes":0,"assignedDepartmentId":30,"transferComment":null,"priorityId":4,"ticketTypeId":5,"userId":105593,"assignedTeamId":71,"assignedAgentId":129239,"respondedById":129239,"resolvedById":129239